<a href="https://colab.research.google.com/github/mehurlock94/analyzing-baltimore-salary-data-and-residency/blob/main/Mini_Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# import tools for the program
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
import numpy as np
import six
from google.colab import files

# import data sets for manipulation
salary_data = pd.read_csv("https://raw.githubusercontent.com/mehurlock94/analyzing-baltimore-salary-data-and-residency/main/Baltimore_Employee_Salaries.csv")
residency_data = pd.read_csv("https://raw.githubusercontent.com/mehurlock94/analyzing-baltimore-salary-data-and-residency/main/Baltimore_City_Employee_Residency_by_Agency.csv")

# clean the date column to calculate the time on the job
variable=salary_data['HireDate'].str.split(' ',1,expand=True)
salary_data['Expand']=variable[0]
variable=salary_data['Expand'].str.split('/')

# at this point, the year, month and date have been extracted into a list that needs to be converted back to a normal date. This normal date is subtracted from the current date to determine days of employment.
dates=[]
for item in variable:
  start = datetime.datetime(int(item[0]),int(item[1]),int(item[2]))
  now = datetime.datetime.today()
  difference = str(now-start)
  difference=difference.split(',')
  difference=difference[0].split(' ')
  dates.append(int(difference[0]))
salary_data["DaysEmployed"]=dates

# remove the unnecessary columns from the final dataframe
remove = ['HireDate','Expand','ID','FirstName','MI','LastName','JobTitle','AgencyID']
for label in remove:
  del salary_data[label]

# pull agency names from the residency data
for col, row in residency_data.iterrows():
  residency_data.at[col,'deptName']=row['deptName'].strip()
residency=pd.unique(residency_data['deptName'])

# set fiscal year to 2020 and clean agency names
salary_data_2020=salary_data[salary_data['FiscalYear']=='FY2020']
variable=salary_data_2020['AgencyName'].str.split('(',1,expand=True)
salary_data_2020['AgencyName']=variable[0]
for col, row in salary_data_2020.iterrows():
  salary_data_2020.at[col,'AgencyName']=row['AgencyName'].strip()
  if 'BPD' in row['AgencyName']:
    salary_data_2020.at[col, 'AgencyName']='Police Department'
  elif 'Police' in row['AgencyName']:
    salary_data_2020.at[col, 'AgencyName']='Police Department'
  elif 'HLTH' in row['AgencyName']:
    salary_data_2020.at[col, 'AgencyName']='HLTH-Health Department'


# pull unique agencies from salary data and create a congruent list between salary data and residency data
salary=pd.unique(salary_data_2020['AgencyName'])
salary.sort()
print (salary)
residency.sort()
joint=[]
for agency in salary:
  if agency in residency:
    joint.append(agency)

# remove unnecessary columns from residency data
residency_data.drop(residency_data.columns.difference(['deptName','totalEmployees','baltCity']),1,inplace=True)
for col, row in residency_data.iterrows():
  total=int(row['totalEmployees'])
  balt=int(row['baltCity'])
  residency_data.at[col, 'percentBalt']=round(balt/total,4)
residency_data=residency_data.rename(columns={'deptName':'AgencyName'})
residency_data=residency_data.set_index('AgencyName')

# determine average salary per agency
salary_data_2020_agg=salary_data_2020.groupby('AgencyName').agg({'AnnualSalary':'mean','GrossPay':'mean','DaysEmployed':'mean'})
salary_data_2020_agg=pd.DataFrame(salary_data_2020_agg.values, index=salary_data_2020_agg.index.values, columns=['AnnualSalary','GrossPay','DaysEmployed'])
salary_data_2020_agg.index.name='AgencyName'

# combine pertinent residency data with salary date
join=pd.merge(residency_data,salary_data_2020_agg,on='AgencyName')
join.index.name=''

# generate plot of employment time vs. percent living in Baltimore
join_melt_1=pd.melt(join,id_vars=['DaysEmployed'],value_vars=['percentBalt'])
graph_1=px.scatter(join_melt_1,x="DaysEmployed",y='value',title='Effect of Employment Time on Living Location',labels={'DaysEmployed':'Days of Employment','value':'Percent of Employees Living in Baltimore'},trendline='ols')
graph_1.write_html('Employment_Time.html')
files.download('Employment_Time.html')

# generate plot of employment time vs. percent living in Baltimore
join_melt_2=pd.melt(join,id_vars=['GrossPay'],value_vars=['percentBalt'])
graph_2=px.scatter(join_melt_2,x="GrossPay",y='value',title='Effect of Gross Pay on Living Location',labels={'GrossPay':'Gross Pay','value':'Percent of Employees Living in Baltimore'},trendline='ols')
graph_2.write_html('Gross_Pay.html')
files.download('Gross_Pay.html')

# horizontal bar plot for balt living data
join=join.sort_values(by=['percentBalt'])
join.drop(join.columns.difference(['percentBalt']),1,inplace=True)
join=join.rename(columns={'percentBalt':'Percent of Employees Living in Baltimore (Lowest 5)'})
join.head()
# join=join.rename(columns={'percentBalt':'Percent of Employees Living in Baltimore (Highest 5)'})
# join.tail()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



['COMP-Audits' 'COMP-Communication Services' "COMP-Comptroller's Office"
 'COMP-Real Estate' 'Circuit Court' 'City Council'
 'Civil Rights & Wage Enforce' 'Council Services' 'DPW-Administration'
 'DPW-Solid Waste' 'DPW-Water & Waste Water' 'Dem 01' 'ERS/EOS Admin'
 'Elections' 'Enoch Pratt Free Library' 'FIN-Acct & Payroll'
 'FIN-Admin & Budgets' 'FIN-Collections' 'FIN-Purchasing'
 'FIN-Risk Management Oper' 'FIN-Treasury Mgt' 'FPR Admin'
 'Fire Department' 'General Services' 'HLTH-Health Department'
 'HR-Human Resources' 'HR-Test Monitor' 'Housing & Community Dev'
 'Law Department' 'Legislative Reference' 'Liquor License Board'
 'M-R Cable & Comm' 'M-R Convention Center' 'M-R Environmental Cntrl'
 'M-R Human Services' 'M-R Info Technology' 'M-R Labor Commissioner'
 "Mayor's Office" 'Mayors Office' 'Municipal & Zoning Appeals 001'
 'OED-Employment Dev' 'Office, Children and Family' "Orphan's Court"
 'Planning Department' 'Police Department' 'R&P-Administration'
 'R&P-Parks' 'R&P-Recrea

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Percent of Employees Living in Baltimore (Lowest 5)
,
Police Department,0.2330
COMP-Audits,0.2973
Fire Department,0.3109
FIN-Acct & Payroll,0.3400
States Attorneys Office,0.4206
